# Lab 9 - Structured Output - GPT-4o-mini

In [ ]:
require 'json'
require 'openai'
require 'dotenv'
require 'date'

Dotenv.load

$client = OpenAI::Client.new(access_token:  ENV['OPENAI_API_KEY'])

In [ ]:
def complete_function(prompt, function)
  resp = $client.chat(
    parameters: {
      messages: [
        {
          role: "system",
          content: "You are a helpful assistant making decisions for routing tasks."
        },
        {
          role: "user",
          content: prompt
        }
      ],
      model: "gpt-4o-mini",
      temperature: 0,
      tools: [
        {
          type: "function",
          function: function
        }
      ]
    }
  )

  begin
    content = resp.dig("choices", 0)

    tool_calls = content.dig("message", "tool_calls")
    return nil unless tool_calls && !tool_calls.empty?
    
    # Get the first tool call
    tool_call = tool_calls[0]
    arguments_str = tool_call.dig("function", "arguments")
    JSON.parse(arguments_str)
  rescue StandardError => e
    puts e.message
    nil
  end
end

## Yes or No (boolean true/false)

In [ ]:
def get_bool_prompt(query)
  <<~PROMPT
    # Instructions

    Is the following question true or false?

    ## Question

    #{query}
  PROMPT
end

In [ ]:
def get_bool_function
  {
    name: "true_or_false",
    strict: true,
    parameters: {
      type: "object",
      properties: {
        final_answer: {
          type: "boolean"
        }
      },
      required: ["final_answer"],
      additionalProperties: false
    }
  }
end

In [ ]:
def truefalse(question)
  prompt = get_bool_prompt(question)
  function = get_bool_function
  response = complete_function(prompt, function)
  if response
    puts "#{response['final_answer']}\t<=\t#{question}"
  end
  response
end

In [ ]:
b = truefalse("Does 2+2 equal 4?")
b = truefalse("Does 2+2 equal 5?")
b = truefalse("Does the text 'Flower City AI conference at the Little Theatre in December' contain an event venue name?")
b = truefalse("Does the text 'Hello, Mr. Smith' list any attendees for an event?")
b = truefalse("Does the text 'Jane Doe is coming to Flower City AI' list any attendees for an event?")
b = truefalse("Does the text 'Jane Doe is coming to the Flower City AI conference' list any attendees for an event?")

## Pick from a list (enum)

In [ ]:
def get_classification_prompt(query)
  <<~PROMPT
    # Instructions

    For an event management SaaS product, natural language queries need to be classified and routed to the appropriate branch. Classify the following user query:

    ## Query

    #{query}
  PROMPT
end

In [ ]:
def get_classification_function
  {
    name: "task_type",
    description: "Classify the task based on the user query.",
    strict: true,
    parameters: {
      type: "object",
      properties: {
        task_type: {
          type: "string",
          enum: ["create_event", "list_events", "create_attendee", "list_attendees", "other"]
        }
      },
      required: ["task_type"],
      additionalProperties: false
    }
  }
end

In [ ]:
def classify(query)
  prompt = get_classification_prompt(query)
  function = get_classification_function
  classification = complete_function(prompt, function)
  if classification
    puts "#{classification['task_type']}\t<=\t#{query}"
  end
  classification
end

In [ ]:
c = classify("What events are coming up?")
c = classify("Book the Ellison Lodge A for December 4th")
c = classify("Jane Doe is coming to the party on 12/4")
c = classify("Pencil in highland park gazebo for june 11th")
c = classify("Leah, Alice, and Fred are all coming on june 11th")
c = classify("Who is coming in June?")
c = classify("What spaces are available?")